In [7]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('./../../designer/script/')
sys.path.append('./../')
sys.path.append('./../../')

from design import Design
from film import FilmSimple
from spectrum import Spectrum
from optimizer.LM_gradient_descent import LM_optimize_d_simple
from optimizer.adam import adam_optimize
import copy
from utils.loss import calculate_RMS
import pickle
import designer
from analyze_utils.structure import plot_layer_thickness


inc_ang = 0.
wls = np.linspace(400, 1000, 500)

In [27]:

def make_random_film(N, pert_budget, seed=1):
    np.random.seed(seed)
    d = np.random.random(N)
    np.random.seed(np.random.randint(100))
    pert_idx = np.random.randint(0, d.shape[0])
    d[pert_idx] += 100
    d *= 3000 / np.sum(d)
    film = FilmSimple('SiO2', 'TiO2', 'Si', d_init=d)
    film.add_spec_param(0., wls)
    film_pert = copy.deepcopy(film)

    d = copy.deepcopy(d) # change reference of d to new
    d[pert_idx] += pert_budget
    film_pert.update_d(d)
    return film, film_pert

def make_pert(dim, rep_num=10, show=False, c='C0', budgets=np.linspace(-100, 100, 20)):

    losses = []
    for rep in range(rep_num):
        if show:
            print(f'dim {dim}, rep {rep}')
        losses.append([])
        for budget in budgets:
            film, film_pert = make_random_film(dim, budget, seed=rep)

            # plot_layer_thickness(film)
            # plot_layer_thickness(film_pert)

            # print(f'before: loss={calculate_RMS(film, film_pert)}')
            adam_optimize(film_pert, [film.get_spec()], 100, alpha=1)
            # print(f'after: loss={calculate_RMS(film, film_pert)}')
            losses[-1].append(calculate_RMS(film, film_pert))
    losses = np.array(losses)
    stds = np.std(losses, axis=0)
    losses = np.mean(losses, axis=0)
    return losses, stds

loss, std = [], []
layer_numbers = [5, 10, 50, 100]
for N in layer_numbers:
    cur_loss, cur_std = make_pert(layer_numbers, show=True, c='C0')
    loss.append(cur_loss)
    std.append(cur_std)


dim 5, rep 0
dim 5, rep 1
dim 5, rep 2
dim 5, rep 3
dim 5, rep 4


KeyboardInterrupt: 

In [ ]:
budgets = np.linspace(-100, 100, 20)

fig, ax = plt.subplots(1, 1)
for cur_loss, cur_std, layer_number, i in zip(loss, std, layer_numbers, range(len(std))):
    ax.plot(budgets, cur_loss, label=f'{layer_numbers} layers', c=f'C{i}')
ax.set_xlabel('pert')
ax.set_ylabel('loss after gd')
ax.legend()

In [25]:
ax.set_ylim(0, None)


(0.0, 0.1677856300692068)